# League of Data - Model Evaluation

En el Notebook anterior, **League of Data - Data Prepropocessing** se realizó todo el proceso de limipieza de la información y se generó un conjunto de datos basado en cocientes para poder comparar dos equipos y así caracterizar una partida.

En este Notebook nos enfocaremos en analizar la estructura del conjunto de datos y encontrar por medio de este, el modelo que mejor prediga cual es el equipo que resultará victorioso en una partida.

Esta meta se alcanzó cumpliendo los siguientes objetivos.

* Visualización del estado de los datos.
* Métodología de validación y Medidas de error.
* Entrenamiento de modelos.
* Analísis de características.
* Selección y extraccion de variables.

### VISUALIZACIÓN DEL ESTADO DE LOS DATOS

Ahora que se tiene el conjunto de datos a entrenar, siempre es una buena ayuda conocer como estan distribuidos los datos, como estos se ven en el espacio o en un plano. El ejercicio que se desarrolló tiene la ventaja de que es un problema de clasificación de tan solo 2 clases, sin embargo, como hemos dicho anteriormente, tiene 33 variables lo cual hace imposible visualizar toda la información sin recurrir a alguna técnica de reducción de dimensiones.

El proceso de reducción de dimensiones previo a la visualización de los datos se realizó con dos técnicas **_Principal component analysis (PCA)_** y **_t-Distributed Stochastic Neighbor Embedding (t-SNE)_**. Como la intención es poder identificar como se distribuyen los datos en el plano y en el espacio, entonces, para cada una de las técnicas se probó reduciendo las características a dos y tres.

**PCA:** Es una técnica no supervisada de extracción de características la cual tiene por objetivo conservar la mayor cantidad de información que sea posible. Lo que hace PCA es realizar una transformación de los datos originales a un nuevo conjunto de varibles, estos conjuntos se ordenan de modo que con unas pocas combinaciones se retengan la mayor cantidad de _variación_ presente en el conjunto **original** de variables.

![PCA 2D](./Images/quotients_pca-2D.png)
Como se puede observar en la gráfica, el conjunto de datos esta muy concentrado en un solo punto, tanto que pareciera que solo existe una clase, lo que nos atreveriamos a decir es que la clasificación va a ser un trabajo bastante complicado.

![PCA 3D](./Images/quotients_pca-3D.png)
Esta gráfica nos confirma aún más el estado en el que se encuentran los datos, la clase 0, correspondiente a las derrotas casi no se logra identificar.

**t-SNE:** Es una técnica no lineal de reducción de dimensiones, en términos muy vagos, **t-sne** lo que hace es formar una distribución de probabilidad sobre los datos, de tal manera que aquellos que son similares tienen una alta probabilidad de pertenecer a un mismo conjunto, mientras que los datos diferentes tienen una probabilidad extremadamente pequeña.

![t-SNE 2D](./Images/quotients_tsne-2D.png)
Con esta gráfica notamos nuevamente que ambas clases presentan un comportamiento similar, todas tienden a juntarse en el mismo lugar y la clase de las victorias intenta sobreponerse a la clase que representa las derrotas, realizar la discriminación va a ser un poco complicado.

![t-SNE 3D](./Images/quotients_tsne-3D.png)
Esta gráfica nos aporta nueva información en comparación a las 3 anteriores. La clase que representa la derrota tiene una mayor presencia, opacando casi por completo a la clase que representa las victorias, sin embargo, podemos notar que la separación de las clases sigue siendo un reto.

De las anteriores gráficas podemos concluir que es muy poco probable obtener resultados de clasificación por encima del 90%, ya que como se nota, la clases estan muy interrelacionadas, hasta el punto de no tener un solo sector en el cual se identifique a solo una de ellas.

Sin embargo, este paso se realiza para que podamos tener una idea de en que estado se encuentran los datos, los resultados definitivos se obtienen a partir del siguiente paso.

### METODOLOGÍA DE VALIDACIÓN Y MEDIDAS DE ERROR

Las métodologías de validación son aquellas estrategías que nos permiten tener una estimación sobre el comportamiento del sistema ante nuevas muestras y con base en esto determinar cual configuración de parámetros es la que mejores resultados nos entrega. En otras palabras, el mejor parámetro no es aquel que mejor se comporta con las muestras del conjunto de datos, sino aquel que mejor se comporta con las muestras desconocidas. 

Dado que cada problema de aprendizaje de máquina cuenta con un conjunto de muestras, las metodologías de validación nos permite separar las datos, de tal manera que se usa una parte para entrenar el modelo y otra para validarlo. Este proceso nos ayuda a obtener buenos resultados estadísticos sobre el desempeño del modelo.

Para efectos del problema que aquí se trata se decidió usar la validación Boostrap, con una distribución de 80% de los datos para entrenar y 20% para validar, siendo un poco más exactos, de las 5426 muestras de cocientes disponibles, 4341 se usarón para entrenar el modelo y 1085 para validarlo.

Dados estos porcentajes, el procedimiento que usa esta metodología es, seleccionar de manera aleatoria las muestras a cada uno de los conjuntos, luego se entrena y valida el modelo para después volver a seleccionar aleatoriamente conjuntos diferentes. Este procedimiento se repetirá 100 veces.

Teniendo en cuenta todo esto, para poder determinar cual modelo es el que mejor se ajusta a nuestro problema, necesitamos contar con una medida que nos indique que tan bueno es, para esto existen las medidas de error.

En un problema de clasificación como el que estamos tratando aquí, el error se da si predecimos que una muestra pertenece a una clase cuando realmente pertenece a otra, esto nos muestra que el error no tiene un grado, sino que el error existe o no existe.

Normalmente, en cualquier problema de aprendizaje de máquina se usa la eficiencia como medida de error, pero en problemas de clasificación es un arma de doble filo porque dada la definición que esta tiene, se asume que las clases poseen igual número de muestras, cuando sabemos que dicha situación no se cumple. Esto permite que la eficiencia quede sesgada a la clase con mayor número de muestras. Por esta razón, es importante tener en cuenta otro tipo de medidas de error, para este problema utilizaremos 4 medidas _sensiblidad, especificidad, eficiencia y presición_.

Para obtener los resultados de estas medidas, recurriremos a una herramienta llamada matriz de confusión, la cual esta definida de la siguiente manera.

![Matriz de confusión](./Images/cm.png)

Por medio de esta herramienta podemos definir las medidas del error. 

**Sensibilidad:** Porcentaje de positivos bien clasificados $\frac{TP}{TP+FN}$.

**Especificidad:** Porcentaje de negativos bien clasificados $\frac{TN}{TN+FP}$.

**Eficiencia:** Porcentaje de aciertos del modelo $\frac{TP+TN}{TP+TN+FP+FN}$.

**Precisión:** Porcentaje de positivos bien predichos $\frac{TP}{TP+FP}$.




### ENTRENAMIENTO DE LOS MODELOS

Después de haber analizado un poco la distribución de los datos, nos dimos cuenta que no se logra percibir un patrón de comportamiento en ellos, lo cual, además de complicarnos la discriminación de las clases, también nos pone difícil la tarea de seleccionar el modelo más pertinente. 

Dado lo anterior se decidió analizar el problema con 8 modelos distintos (Regresión Lineal, Ventana de Parzen, KNN, Random Forest, XGBoost, Redes Neuronales, Redes Neuronales Profundas, Máquinas de Soporte Vectorial), cada uno de ellos tiene una serie de hiperparámetros que dependiendo de su configuración presentan resultados diferentes. Por lo tanto, se ha establecido una malla de valores en cada modelo. 

Para cada configuración de hiperparámetros en su respectivo modelo, se realizó la misma metodología de validación con iguales condiciones, esto con el fin tener un punto en común de evaluación.

Dado que algunos modelos cuentan con una cantidad considerable de configuraciones para sus hiperparámetros, los resultados que se expondrán en este Notebook serán aquellos que presenten el mejor resultado. Estos resultados se mostrarán por medio de dos gráficas,  La Curva ROC y el consolidado de las medidas de error.

* **Regresión Linear**

Es considerado el modelo más elemental dentro del aprendizaje de máquina, sin embargo, se ha demostrado que bajo ciertos escenarios presenta unos grandes resultados. 

El hiperparámetro que se tuvo en cuenta para este modelo es el grado del polinomio y los valores con que se probó fueron: _polinomio grado 1 y 2_, siendo el polinomio de grado 1 la mejor configuración para este modelo.

| Curva ROC                                  | Medidas de Error                                     |
| :----------------------------------------: | :--------------------------------------------------: |
| ![logistic_ROC](./Images/logistic_ROC.png) | ![logistic_Measures](./Images/logistic_Measures.png) |


* **k-Nearest Neighbors (KNN)**

Este es un modelo que evalua todas las muestras que contiene el conjunto de datos y con base en alguna medida de similaridad predice para un valor para una nueva muestra. Para los problemas de clasificación como este, se evalua cuantas muestras son las más cercanas a la nueva y por medio de la _moda_ se identifica a cual clase pertenece.

Los hiperparámetros con los que cuenta este modelo son el número de vecinos cercanos y la medida de distancia. Para el primero se tomó en cuenta los valores 1,2,3,4,5,10 y 100; en cuanto al segundo, solo se tomó en cuenta una configuración y fue la distancia euclidiana.

El mejor resultado se obtuvo con la configuración de 100 vecinos.

| Curva ROC                                  | Medidas de Error                                     |
| :----------------------------------------: | :--------------------------------------------------: |
| ![knn_ROC](./Images/knn_ROC.png) | ![knn_Measures](./Images/knn_Measures.png) |


* **Ventana de Parzen**

Este modelo es muy similar al anteriormente mencionado (KNN), la diferencia es que en este modelo no se da la definición de "_vecindad_" (Área variable definida por los puntos más cercanos), sino que por medio de una función Kernel genera una función de densidad de probabilidad. 

El hiperparámetro que se configura en este modelo es el ancho de la ventana, en otras palabras se modifica el peso que se le asigna a las muestras, entre más cerca, más peso asigna y mientras más lejos menos peso. La malla de valores asignada fue 0.05, 0.1, 0.5, 1, 10.

El mejor resultado se dío con la ventana de ancho 0.5.

![parzenwindow_Measures](./Images/parzenwindow_Measures.png)


##### Comité de máquinas

Una práctica muy usada en el aprendizaje de máquina son los _comités de máquinas_, por medio de este concepto se busca que la responsabilidad de decición de un modelo no se limite unicamente a un estimador, sino que se combine la decisión de varios estimadores para obtener de allí la solución definitiva. Dos de las técnicas más conocidas son el **Bagging** y el **Boosting**. 

La primer técnica lo que hace es, tomar el conjunto de entrenamiento  y realizar una serie de muestreos sobre el, luego, entrena un clasificador diferente para cada uno de los conjuntos anteriormente seleccionados. La decisión final se toma dependiendo de si el problema es de clasificación o de regresión, para un problema de clasificación se utiliza la moda, mientras que para un problema de regresión se usa el promedio.
 
La segunda técnica consiste en tomar todo el conjunto de entrenamiento y con él entrenar un modelo, después de esto se seleccionan aquellas muestras que quedaron mal interpretadas (estas muestras se les conoce como muestras pesadas) mas algunas otras muestras (tomadas aleatoriamente del conjunto de entrenamiento) y con ellas se vuelve a entrena el modelo.

Para el problema que estamos desarrollando se tuvo en cuenta un modelo que implementa la técnica de Bagging y otro que implementa la técnica de Boosting.

* **Random Forest**

Este modelo es un comité de árboles de decisión el cual usa la técnica de Bagging con un característica adicional. La diferencia fundamental es que en la técnica normal de Bagging, se entrenan muestras aleatroias pero todas con las mismas variables, mientras que en el Random Forest las muestras son aleatorias para cada árbol y además las características que se evaluan en los respectivos árboles también se escogen al azar.

El hiperparámetro que se evalua en este modelo es la cantidad de árboles de decisión en cada estimador, la malla de valores escogida fue 10, 20, 30, 40, 50 y 500 árboles.

El mejor resultado para este modelo se consiguió con la configuración de 500 árboles.

| Curva ROC                                  | Medidas de Error                                     |
| :----------------------------------------: | :--------------------------------------------------: |
| ![forest_ROC](./Images/forest_ROC.png) | ![forest_Measures](./Images/forest_Measures.png) |


* **XGBoost**

Como el modelo anterior, este modelo también es un comité de árboles de decisión pero en este caso usa la técnica de Boosting. 

Para este modelo, el hiperparámetro a configurar y la malla de valores son iguales, el hiperparámetro es la cantidad de árboles y la malla de valores fue 10, 20, 30, 40, 50 y 500 árboles.

El mejor resultado se encontró con la configuración de 40 árboles

| Curva ROC                                  | Medidas de Error                                     |
| :----------------------------------------: | :--------------------------------------------------: |
| ![xgboost_ROC](./Images/xgboost_ROC.png) | ![xgboost_Measures](./Images/xgboost_Measures.png) |


##### Redes Neuronales

* **MLP**

El perceptrón multicapa (MLP) es un tipo de red neuronal artificial que utiliza como unidad básica y principal al perceptrón. Este elemento no es más que un clasificador lineal que toma valores reales como entrada, calcula una combinación lineal de estas entradas y produce una salida, 1 si el resultado es mayor que algún umbral y -1 en otro caso.

En general, las redes neuronales tienen algo que se conoce como capas, estas capas se distribuyen en:
* Entrada: Recibe las características o variables a ser evaluadas.
* Ocultas: Capas de perceptrones.
* Salida: Donde se entregan los resultados.

Una de las principales características de la red MLP es que siempre los perceptrones de una capa estan conectados completamente a todos los perceptrones de la siguiente capa. 

Otra característica muy importante, es que la red se entrena hacia adelante (FeedForward) pero se propaga hacía atras (BackPropagation) y siempre lo hacen pasando por cada una de las capas anteriormente mencionadas.

Para este modelo de perceptrón multicapa se configuraron tres hiperparámtros. El primero es el número de iteraciones al cual se le dió un valor constante de 500. El segundo es el número de capas ocultas el cual esta en 1, 2 y 3 capas. El tercer hiperparámtro es el número de neuronas por capa oculta, la malla de valores se explica en la siguiente tabla.

| Neuronas en la capa 1 | Neuronas en la capa 2 | Neuronas en la capa 3 |
| :-------------------: | :-------------------: | :-------------------: | 
| 16 | - | - |
| 32 | - | - |
| 48 | - | - |
| 64 | - | - |
| 16 | [4,8,12] | - |
| 32 | [10,20,30] | - |
| 48 | [15,30,45] | - |
| 64 | [20,40,60] | - |
| 16 | [4,8,12] | [2,3],[3,6],[5,10] |
| 32 | [10,20,30] | [4,8],[9,18],[14,28] |
| 48 | [15,30,45] | [6,12],[14,28],[21,42] |
| 64 | [20,40,60] | [9,18],[16,32],[29,58] |

El mejor resultado, se obtuvo con la configuración de 3 capas ocultas y 32 neuronas en la primer capa, 20 neuronas en la segunda y 9 neuronas en la tercera.

| Curva ROC                                  | Medidas de Error                                     |
| :----------------------------------------: | :--------------------------------------------------: |
| ![mlp_ROC](./Images/mlp_ROC.png) | ![mlp_Measures](./Images/mlp_Measures.png) |


* **Deep NN**

Este modelo es muy usado en la actualidad debido a que los costos de ejecución se han acortado, entre otras cosas, gracias al creciente uso de las GPU. La diferencia principal con una red neuronal tradicional tipo MLP (como el modelo anteriormente descrito) es que las redes neuronales profundas usan una enorme cantidad de capas ocultas, además dichas capas pueden ser de diferentes tipos como _Densas, combolucionales, activación, entre otras_.

Para este modelo no se configuró una malla de valores, los hiperparámetros que se tuvieron en cuenta solo tomaron un valor. El número de capas ocultas fue 9, el tipo de capa oculta fue _Densa_ (completamente conectada), la función de activación en cada capa fue la función _relu_, el número de neuronas por capas oculta fue 32, 28, 24, 20, 16, 12, 8, 4 y 1 respectivamente y finalmente el número de épocas fue 500.

Los resultados obtenidos para las medidas de error fueron los siguientes.
![deepNN_Measures](./Images/deepNN_Measures.png) |


##### Máquinas de Soporte Vectorial

Un concepto que es muy importante definir es la diferencia entre el riesgo estructural y el riesgo empírico. El primer tipo de riesgo consiste en elegir la mejor frontera de decisión para la cual la probabilidad de error a futuro sea menor, osea, es encontrar la mejor frontera de entre todas las posibles en la cual el error con las muestras de validación sea mínimo. Por otra parte el riesgo empírico se trata sobre medir el error con base en el conjunto de entrenamiento y esperar que haya un comportamiento similar o igua en el conjunto de validación.

Este modelo se basa en el riesgo estructural porque nos permite encontrar la mejor solución a un problema utilizando como criterio de ajuste la maximización del márgen. El márgen es la distancia mas corta entre la frontera de decisión y cualquier muestra de nuestro conjunto de datos. En conclusión una máquina de soporte vectorial siempre buscará la frontera que mejor separe las muestras.

Pero entonces **¿Qué es un vector de soporte?** Los vectores de soporte son todas aquellas muestras que tocan el límite del márgen de decisión, aquellas muestras con un grado de incertidumbre muy alto, las difíciles de clasificar. Con estas muestras complicadas es que se crea la frontera de decisión.

Para este problema se usarón máquinas de soporte vectorial con dos configuraciones diferentes, una con kernel lineal y la otra con kernel radial. La primer implementación es similar al modelo de regresión logistica, mientras que la segunda implementación es similar al modelo de ventana de parzen (por el tipo kernel que se usa).

* **Kernel Lineal**

El hiperparámtro que se configuró para este modelo fue el Box Constraint y la malla de valores que se le asignó esta basado en potencias de 10, empezando desde 0.01 y terminando en 100.

El mejor resultado que se obtuvo fue con un valor de 0.01 para el Box Constraint.

| Curva ROC                                  | Medidas de Error                                     |
| :----------------------------------------: | :--------------------------------------------------: |
| ![svmLinear_ROC](./Images/svmLinear_ROC.png) | ![svmLinear_Measures](./Images/svmLinear_Measures.png) |

* **Kernel RBF**

Para esta configuración de kernel radial se configuraron dos hiperparámetros, el primero fue el Box Constraint y el segundo el valor gamma, este cumple un papel similar al hiperparámetro configurado en el modelo de ventana de parzen. Nuevamente la malla de valores esta basada en potencias de 10 empenzando desde 0.01 y terminando en 100, tanto para el Box Constraint como para el gamma. Es importante aclarar que para cada valor del Box Constraint el hiperparámetro gamma toma los 5 valores anteriormente mencionados.

| Box Constraint | Gamma | 
| :------------: | :---: |
| 0.01 | 0.01|
| 0.01 | 0.1 |
| 0.01 | 1   |
| 0.01 | 10  |
| 0.01 | 100 |
| ---- | --- |
| 100 | 0.01|
| 100 | 0.1 |
| 100 | 1   |
| 100 | 10  |
| 100 | 100 |

El mejor resultado que se obtuvo fue con un valor de 1 para el Box Constraint y 10 para Gamma.

| Curva ROC                                  | Medidas de Error                                     |
| :----------------------------------------: | :--------------------------------------------------: |
| ![svmGaussian_ROC](./Images/svmGaussian_ROC.png) | ![svmGaussian_Measures](./Images/svmGaussian_Measures.png) |



Luego de haber usado cada uno de los modelos, podemos confirmar la hipotesis que nos planteamos en la etapa de analisis de distribución de los datos, no fue posible en ninguna de las medidas de error alcanzar el 90% de aciertos, de hecho, tampoco se alcanzó el 80% de aciertos.

Otros factores que también se pueden interpretar son.

* El porcentaje de positivos bien clasificados (Sensibilidad) es la medida que más fluctuación tiene, el modelo que peor predijó esta medida entregó un 51% de aciertos, mientras que el modelo con mejor resultado,  entregó un 78% de aciertos.
* La especificidad fue la medida con peores resultados, la mayor parte de los modelos entregó valores entre el 50% y el 55%, solo para un modelo se obtuvo un resultado superior al promedio (76%). Esto resultados nos quieren decir que las muestras de la clase que representa las derrotas en una partida son muy dificiles de clasificar, en otras palabras, al sistema le da más dificultad determinar cuando un equipo será derrotado.
* La exactitud es la medida de error más constante en todos los modelos, la mayor parte de las ejecuciones se obtuvieron resultados entre 65% y 68%.
* La presición es la medida de error con mejores resultados, siempre por encima del 66% con un tope máximo del 77%. Lo que podemos concluir de esto es que el sistema tiene una gran capacidad para predecir la clase que representa las victorias, porque muy pocas veces da como ganador a un equipo que perdió.
* Se puede concluir que los modelos que mejores resultados entregan son el XGBoost y la red neuronal profunda. Cada uno tiene falencias en una medida de error, el XGBoost tiene problemas con la especificidad, mientras que la red neuronal tiene problemas con la sensibilidad, sin embargo en el resto de medidas entregan muy buenos resultados. 
    * Si se escogiera un solo modelo como ganador, este, sería el XGBoost, porque apesar de tener resultados similares a la red neuronal, este tiene mejores tiempos de ejecución.
    * No es de extrañar que estos dos modelos entreguen los mejores resultados, si se hace una revisión del estado actual de las competencias de Machine Learning, encontramos que en la mayorias de los casos estos modelos son los más utilizados y también los que mejores resultados entregan.
    
